In [3]:
# First, import necessary dependencies

import torch 
import torch.nn as nn
import torch.optim as optim
from torchtext import data
import pandas as pd
import numpy as np
import random
import spacy

In [29]:
# Define path, device

PATH = './data'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
# get texts for classifier path 
# texts are organized into folders of classes
def get_texts_class(path, classes, train_pct):
    texts, labels = [], []
    for c in classes: 
        for fname in (path/c).glob('*.*'):
            texts.append(fname.open('r', encoding='utf-8').read())
            labels.append(classes.index(c))
    texts, labels = np.array(texts), np.array(labels)
    cols = ['labels', 'text']
    df = pd.DataFrame({'text': texts, 'labels': labels}, columns=cols)
    rand_idx = np.random.permutation(len(df))
    df = df.loc[rand_idx]
    train_df = df.iloc[:int((train_pct * len(df)))]
    val_df = df.iloc[int((train_pct * len(df))):]   
    return train_df, val_df


def manual_rand(df, train_pct):
    rand_idx = np.random.permutation(len(df))
    df = df.loc[rand_idx]
    train_df = df.iloc[:int((train_pct * len(df)))]
    val_df = df.iloc[int((train_pct * len(df))):]   
    return train_df, val_df



In [21]:
# Read dataframe, isolate cols
data_df = pd.read_csv('train.csv', encoding='latin-1')
data_df = data_df[['Sentiment', 'SentimentText']]

# create toy df
toy_df = data_df[:1000]
train_df, valid_df = manual_rand(toy_df, 0.7)

In [22]:
valid_df.head()

,Sentiment,SentimentText
200,0,@georgediaz #Magic ..thinking less than 50 % ...
519,0,I am actually crying now because of sore thr...
755,1,@DWO34 Great to follow @BUTTERFLYWHEEL @Debra...
907,0,Dont Wanna Leave NY Today.
30,0,I hate it when any athlete appears to tear ...


In [23]:
train_df.to_csv(f'{PATH}/train.csv', index=False, header=False)
test_df.to_csv(f'{PATH}/valid.csv', index=False, header=False)

In [24]:
# Define fields

TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField(dtype=torch.float)

fields = [
    ('label', LABEL),
    ('text', TEXT)
]

# Create datasets
train_data, valid_data = data.TabularDataset.splits(path=PATH,
                                                    train='train.csv',
                                                    test ='valid.csv',
                                                    format='csv',
                                                    fields=fields)

In [25]:
print(vars(valid_data.examples[71]))

{'label': '0', 'text': ['-', '@littlecharva', '-', 'you', "'ll", 'be', 'pleased', 'to', 'know', 'I', 'did', 'not', 'have', 'a', 'nice', 'night', "'s", 'sleep', ',', 'even', 'in', 'a', 'comfy', 'bed', ';)', 'even', 'a', 'sleeping', 'bag', 'would', 'be', 'same']}


In [27]:
# Build the vocab using GloVE vectors, 100 dim

TEXT.build_vocab(train_data, max_size=25000, vectors='glove.6B.100d')
LABEL.build_vocab(train_data)
print(f'training data length: {len(train_data)}')
print(f'training vocab length: {len(TEXT.vocab)}')
print(f'label vocab length: {len(LABEL.vocab)}')

.vector_cache/glove.6B.zip: 862MB [03:18, 4.34MB/s]                               
100%|█████████▉| 398797/400000 [00:18<00:00, 22803.84it/s]

training data length: 700
training vocab length: 3154
label vocab length: 3154


100%|█████████▉| 398797/400000 [00:30<00:00, 22803.84it/s]

In [30]:
# Create dataset iterators

BATCH_SIZE = 64

train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_data, valid_data),
    batch_size = BATCH_SIZE,
    device=device,
    sort=False)

In [33]:
# Build the model 

class biLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim,
                n_layers, bidirectional, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers,
                          bidirectional=bidirectional, dropout=dropout)
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        embedded = self.dropout(self.embedding(x))
        output, (hidden, cell) = self.rnn(embedded)
        #concatenate final forward, backward layers
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))        
        return self.fc(hidden.squeeze(0))

In [91]:
# set structural parameters

VOCAB_SIZE = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 5
BIDIRECTIONAL = True
DROPOUT = 0.5

model = biLSTM(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
           N_LAYERS, BIDIRECTIONAL, DROPOUT)

In [92]:
# load the pretrained GloVE embeddings
pretrained_embeddings = TEXT.vocab.vectors

# replace the weights of the model embedding layer with pretrained embeddings
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [93]:
# set loss function
criterion = nn.BCEWithLogitsLoss()

# set optimizer to Adam
optimizer = optim.Adam(model.parameters())

In [94]:
# set GPU
model = model.to(device)
criterion = criterion.to(device)

In [95]:
# define accuracy function
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds)).float()
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum()/len(correct)
    return acc

In [96]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    # keeps dropout on
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label.float())
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [97]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    # turns dropout off
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label.float())
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [98]:
N_EPOCHS = 15

for epoch in range(N_EPOCHS):

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}% |')

| Epoch: 01 | Train Loss: 0.666 | Train Acc: 65.84% | Val. Loss: 0.646 | Val. Acc: 67.64% |
| Epoch: 02 | Train Loss: 0.638 | Train Acc: 65.71% | Val. Loss: 0.608 | Val. Acc: 67.64% |
| Epoch: 03 | Train Loss: 0.614 | Train Acc: 66.33% | Val. Loss: 0.583 | Val. Acc: 70.00% |
| Epoch: 04 | Train Loss: 0.581 | Train Acc: 71.20% | Val. Loss: 0.523 | Val. Acc: 73.30% |
| Epoch: 05 | Train Loss: 0.558 | Train Acc: 72.98% | Val. Loss: 0.543 | Val. Acc: 71.31% |
| Epoch: 06 | Train Loss: 0.522 | Train Acc: 76.71% | Val. Loss: 0.503 | Val. Acc: 78.41% |
| Epoch: 07 | Train Loss: 0.456 | Train Acc: 80.49% | Val. Loss: 0.462 | Val. Acc: 78.86% |
| Epoch: 08 | Train Loss: 0.468 | Train Acc: 79.47% | Val. Loss: 0.421 | Val. Acc: 81.19% |
| Epoch: 09 | Train Loss: 0.430 | Train Acc: 81.88% | Val. Loss: 0.526 | Val. Acc: 76.11% |
| Epoch: 10 | Train Loss: 0.430 | Train Acc: 81.43% | Val. Loss: 0.401 | Val. Acc: 81.68% |
| Epoch: 11 | Train Loss: 0.391 | Train Acc: 83.99% | Val. Loss: 0.370 | Val. Ac

In [106]:
# Obtain the result for a single input

nlp = spacy.load('en')

def predict_class(input):
    # tokenize the input
    tokenized = [tok.text for tok in nlp.tokenizer(input)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    prediction = torch.sigmoid(model(tensor))
    pred = torch.round(prediction)
    if pred == 0: print('negative')
    else: print('positive')
    return prediction.item()

In [130]:
predict_class("suck a dick bill gates")

negative


0.04189132899045944

In [129]:
predict_class("I'm just really happy")

positive


0.6655281186103821